In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, SGD

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import load_model 

import numpy as np
import pickle
import time

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
datagen = ImageDataGenerator(rescale=1./255)


train_generator = datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/dataset/data_2categ/data_augmented/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode="binary",
    shuffle=True,
    seed=7
)

valid_generator = datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/dataset/data_2categ/data_augmented/validation",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode="binary",
    shuffle=True,
    seed=7
)

Found 4864 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


In [3]:
base_model = InceptionV3(weights=None, include_top=False, input_shape = (256,256, 3))

In [4]:
NAME = "dimentia-inception-CNN-{}".format(int(time.time()))

model = Sequential()

model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))
    
model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
activation_94 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [5]:
def lr_schedule(epoch):
    
    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 40: # 120
        lr *= 1e-2
    elif epoch > 20: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [6]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/CNN_pretrained/logs/{}".format(NAME))

#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7)
#filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint('best_model_inception.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'],
              )


Learning rate:  0.0001


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[lr_scheduler, checkpoint, tensorboard])

Instructions for updating:
Please use Model.fit, which supports generators.
Learning rate:  0.0001
Epoch 1/50
  1/304 [..............................] - ETA: 0s - loss: 0.6373 - accuracy: 0.6250WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
304/304 [==============================] - ETA: 0s - loss: 0.6771 - accuracy: 0.5989
Epoch 00001: val_accuracy improved from -inf to 0.56250, saving model to best_model_inception.h5
304/304 [==============================] - 50s 166ms/step - loss: 0.6771 - accuracy: 0.5989 - val_loss: 0.7880 - val_accuracy: 0.5625
Learning rate:  0.0001
Epoch 2/50
304/304 [==============================] - ETA: 0s - loss: 0.5648 - accuracy: 0.7039
Epoch 00002: val_accuracy did not improve from 0.56250
304/304 [====================

Learning rate:  1e-05
Epoch 24/50
304/304 [==============================] - ETA: 0s - loss: 0.0019 - accuracy: 0.9998
Epoch 00024: val_accuracy did not improve from 0.69643
304/304 [==============================] - 49s 161ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 2.6147 - val_accuracy: 0.6339
Learning rate:  1e-05
Epoch 25/50
  1/304 [..............................] - ETA: 0s - loss: 7.2173e-05 - accuracy: 1.0000

In [ ]:
model = load_model('best_model_inception.h5')

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)